In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [67]:
#Read final dataset for work
path="/home/dmacs/Downloads/Mini_Project/Beth_dataset/labelled_training_data.csv"  # Path is the dataset path. copy path from the drive and paste here
df=pd.read_csv(path,low_memory=False)

df1 = pd.read_csv("/home/dmacs/Downloads/Mini_Project/Beth_dataset/labelled_testing_data.csv")
df2 = pd.read_csv("/home/dmacs/Downloads/Mini_Project/Beth_dataset/labelled_validation_data.csv")

df = df.append(df1)
df = df.append(df2)

/tmp/ipykernel_42354/2995610886.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df1)
/tmp/ipykernel_42354/2995610886.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2)


In [68]:
df.columns


Index(['timestamp', 'processId', 'threadId', 'parentProcessId', 'userId',
       'mountNamespace', 'processName', 'hostName', 'eventId', 'eventName',
       'stackAddresses', 'argsNum', 'returnValue', 'args', 'sus', 'evil'],
      dtype='object')

In [77]:
evil = df[df['evil']==1]
normal = df[df['evil']==0]
print("Evil Records:\n")
# pd.DataFrame(evil)['mountNamespace'].nunique()
pd.DataFrame(normal)['mountNamespace'].nunique()

# print("\n\nNormal Records:\n")
# pd.DataFrame(normal)


Evil Records:



6

In [34]:

df_with_evil = df.copy()


In [35]:
df = pd.DataFrame(df[["hostName","processId", "parentProcessId", "userId", "mountNamespace", "eventId", "argsNum", "returnValue","sus"]])


In [36]:
df.head()


,hostName,processId,parentProcessId,userId,mountNamespace,eventId,argsNum,returnValue,sus
0,ip-10-100-1-120,381,1,100,4026532231,157,5,0,1
1,ip-10-100-1-120,381,1,100,4026532231,3,1,0,1
2,ip-10-100-1-120,381,1,100,4026532231,1010,0,0,1
3,ip-10-100-1-120,7347,7341,0,4026531840,21,2,-2,1
4,ip-10-100-1-120,7347,7341,0,4026531840,1005,4,0,1


In [37]:
#Target column encoding
from sklearn import preprocessing

def encode_text_index(df,name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

In [38]:
le = preprocessing.LabelEncoder()
le.fit_transform(df['hostName'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

{'ip-10-100-1-120': 0, 'ip-10-100-1-129': 1, 'ip-10-100-1-165': 2, 'ip-10-100-1-169': 3, 'ip-10-100-1-173': 4, 'ip-10-100-1-217': 5, 'ip-10-100-1-28': 6, 'ip-10-100-1-34': 7, 'ip-10-100-1-55': 8, 'ip-10-100-1-57': 9, 'ip-10-100-1-79': 10, 'ubuntu': 11}


In [39]:
def process_data(unprocessed_data):
    data = unprocessed_data.copy()
    data["processId"] = data["processId"].map(lambda x: 0 if x in [0, 1, 2] else 1)  # Map to OS/not OS
    data["parentProcessId"] = data["parentProcessId"].map(lambda x: 0 if x in [0, 1, 2] else 1)  # Map to OS/not OS
    data["userId"] = data["userId"].map(lambda x: 0 if x < 1000 else 1)  # Map to OS/not OS
    data["mountNamespace"] = data["mountNamespace"].map(lambda x: 0 if x == 4026531840 else 1)  # Map to mount access to mnt/ (all non-OS users) /elsewhere
    data["eventId"] = data["eventId"]  # Keep eventId values (requires knowing max value)
    data["returnValue"] = data["returnValue"].map(lambda x: 0 if x == 0 else (1 if x > 0 else 2))  # Map to success/success with value/error

    return data

In [40]:
df = process_data(df)


In [41]:
#Encoding the numeric column
def  encode_numeric_zscore(df, name, mean=None, sd=None):
  if mean is None:
    mean= df[name].mean()

  if sd is None:
    sd=df[name].std()

  df[name] = (df[name] - mean) / sd

In [42]:
# encode_numeric_zscore(df,'mountNamespace')
# encode_numeric_zscore(df,'threadId')
# encode_numeric_zscore(df,'processId')
# encode_numeric_zscore(df,'timestamp')
# encode_numeric_zscore(df,'parentProcessId')
# encode_numeric_zscore(df,'eventId')
# encode_numeric_zscore(df,'returnValue')

In [43]:
hostNmae = encode_text_index(df,'hostName')
df.head()

,hostName,processId,parentProcessId,userId,mountNamespace,eventId,argsNum,returnValue,sus
0,0,1,0,0,1,157,5,0,1
1,0,1,0,0,1,3,1,0,1
2,0,1,0,0,1,1010,0,0,1
3,0,1,1,0,0,21,2,2,1
4,0,1,1,0,0,1005,4,0,1


In [44]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(0, inplace=True)

In [45]:
# Convert to numpy - Classification
# x_columns = df.columns.drop('evil')
x_columns = df.columns
x = df[x_columns].values

dummies = pd.get_dummies(df_with_evil['evil']) # Classification
outcomes = dummies.columns
num_classes = len(outcomes)

y = dummies.values

In [46]:
# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, shuffle = True, random_state=64)

In [47]:
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 128)
        self.fc4 = nn.Linear(128, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        x = self.softmax(x)
        return x


In [60]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs, patience=5):
    best_loss = float('inf')
    best_model = None
    early_stopping_counter = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}")

        # Validate
        val_loss, _ = evaluate_model(model, criterion, val_loader)

        # Check for early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            best_model = model.state_dict().copy()
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= patience:
            print("Early stopping triggered. No improvement in validation loss for {} epochs.".format(patience))
            break

    return best_model



def evaluate_model(model, criterion, dataloader):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    accuracy = correct_predictions / total_predictions

    print(f"Validation Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")

    return epoch_loss, accuracy


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# Define the model architecture
class MyModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 128)
        self.fc4 = nn.Linear(128, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        x = self.softmax(x)
        return x

# Function to train the model
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs, patience=5):
    best_loss = float('inf')
    best_model = None
    early_stopping_counter = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}")

        # Validate
        val_loss, _ = evaluate_model(model, criterion, val_loader)

        # Check for early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            best_model = model.state_dict().copy()
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= patience:
            print("Early stopping triggered. No improvement in validation loss for {} epochs.".format(patience))
            break

    return best_model

# Function to evaluate the model
def evaluate_model(model, criterion, dataloader):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    accuracy = correct_predictions / total_predictions

    print(f"Validation Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")

    return epoch_loss, accuracy

# Load the dataset
# Split the dataset into train, validation, and test sets
x_train, x_val_test, y_train, y_val_test = train_test_split(x, y, test_size=0.2)
x_val, x_test, y_val, y_test = train_test_split(x_val_test, y_val_test, test_size=0.5)

# Convert data to PyTorch tensors
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.argmax(axis=1), dtype=torch.int64)  # Assuming y_train is one-hot encoded
x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.argmax(axis=1), dtype=torch.int64)  # Assuming y_val is one-hot encoded
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.argmax(axis=1), dtype=torch.int64)  # Assuming y_test is one-hot encoded

# Create DataLoader for train, validation, and test sets
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=256)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=256)

input_dim = x_train.shape[1]
output_dim = y_train.shape[1]

# Define the model
model = MyModel(input_dim, output_dim)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)  # Corrected learning rate parameter name

# Train the model
num_epochs = 5
best_model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs, patience=5)

# Evaluate the best model on the test set
evaluate_model(model, criterion, test_loader)

# Save the best model
torch.save(best_model, 'best_model.pth')


In [ ]:

import torch.optim as optim

from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# Split the dataset into train, validation, and test sets
x_train, x_val_test, y_train, y_val_test = train_test_split(x, y, test_size=0.2)
x_val, x_test, y_val, y_test = train_test_split(x_val_test, y_val_test, test_size=0.5)

# Convert data to PyTorch tensors
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.argmax(axis=1), dtype=torch.int64)  # Assuming y_train is one-hot encoded
x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.argmax(axis=1), dtype=torch.int64)  # Assuming y_val is one-hot encoded
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.argmax(axis=1), dtype=torch.int64)  # Assuming y_test is one-hot encoded

# Create DataLoader for train, validation, and test sets
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=256)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=256)

input_dim = x_train.shape[1]
output_dim = y_train.shape[1]

# Define the model
model = MyModel(input_dim, output_dim)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001)

# Train the model
num_epochs = 50
best_model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs, patience=5)

# Evaluate the best model on the test set
evaluate_model(best_model, criterion, test_loader)

# Save the best model
torch.save(best_model.state_dict(),  'best_model.pth')
# save_model(best_model, 'best_model.pth')
